# Model integration

This tutorial demonstrate how an openalea model can be exported as an IPM-compliant web service

## Python Model creation

Let export the folowwing simple model, that return some risk level, as a function of air temperature and a user-defined temperature threshold:

In [2]:
def t_risk(tair, threshold=15):
    if tair <= threshold:
        return 0
    else:
        return 1

In [3]:
t_risk(0), t_risk(20), t_risk(20, threshold=20)

(0, 1, 0)

## Import in OpenAlea

Let import it in OpenAlea, by embending it in a OpenAlea Node:

In [4]:
from openalea.core.node import FuncNode
from openalea.core import IFloat, IInt

In [5]:
    inputs = (dict(name='tair', interface=IFloat, value=None),
             dict(name='threshold', interface=IFloat, value=15))
    outputs = (dict(name='Risk', interface=IInt), )
    my_node = FuncNode(inputs, outputs, t_risk)
    my_node.name='TRISK'

In [6]:
my_node((0,)), my_node((20,)),my_node((20,20))

(0, 1, 0)

In [7]:
my_node.name, my_node.input_desc, my_node.output_desc

('TRISK',
 [{'name': 'tair', 'interface': IFloat, 'value': None},
  {'name': 'threshold', 'interface': IFloat, 'value': 15}],
 [{'name': 'Risk', 'interface': IInt}])

## Create IPM model

In [8]:
from openalea.dss import Manager
from openalea.dss.dss_factory import dss_factory
help(dss_factory)

Help on function dss_factory in module openalea.dss.dss_factory:

dss_factory(node, interval=86400, weather_parameters=None, parameters=None, template=None)
    Transform an openalea node in a IPM model json descriptor and generate a fastAPI script to launch webservice
    
    Args:
        node: the node to be exported
        interval: the time step of the model (s)
        weather_parameters: a mapping between node input name and weather data codes, if any.
        None if none of the input is a weather data
        parameters: a list of node input name to be exposed as config parameters in IPM-Decison platform
        template: (optional) an existing IPM model to be used as a template for filling missing information
    Returns:
        model: a json-like dict describing the model
        dss_service: a string containing the script to be run for launching the web service



In [9]:
m = Manager()
template = m.get_model("no.nibio.vips","PSILARTEMP")

In [12]:
model, service = dss_factory(my_node, weather_parameters={'tair': 1002},template=template)

In [13]:
model

{'name': 'Carrot fly flight model',
 'id': 'TRISK',
 'version': '1.0',
 'purpose': 'Estimates risk of flight and egg laying in crop',
 'description': 'THE PEST: The first generation of adult carrot fly emerge from pupae in the soil in the spring, and lay eggs close to the base of vulnerable crops. Larvae initial feed at the surface, then tunnel into the tap root. Adults emerge mid-July and can lead to a second generation. \nTHE DECISION: Treatments may need to be applied soon after adults arrive in the crop, before larvae tunnel into the crop roots.  \nTHE MODEL: The model determines the start of the flight period for the 1st generation of carrot rust fly based on accumuleted degree-days (260 day-degrees) over a base temperature of 5°C.  \nTHE PARAMETERS: The model uses daily air temperature \nSOURCE: Luke, Finland. \nASSUMPTIONS: Be aware that in areas with field covers (plastic, single or double non-woven covers, etc.) with early crops the preceding season (either on the current fiel

In [15]:
print(service)

from fastapi import FastAPI

app = FastAPI()

@app.post("dss/model/no.nibio.vips/TRISK")
async def model_evaluation(item):
    return item

